In [2]:
import pandas as pd
df = pd.read_csv('car_price_prediction.csv.zip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

In [3]:
df.drop(['Doors', 'Wheel', 'ID', 'Leather interior', 'Levy'], axis=1, inplace=True)
# keraksiz ustunlar o'chirildi , chunki ulardagi ma'lumot regressiyani ishlashiga halaqit beradi

In [4]:
def convert(value):
    return float(value)
df['Airbags'] = df['Airbags'].apply(convert)
df['Prod. year'] = df['Prod. year'].apply(convert)

In [5]:
def convert(value):
    if 'Turbo' in value:
        value = value.replace('Turbo',' ')
        return float(value)
    elif 'km' in value:
        value = value.replace('km',' ')
        return float(value)
    elif ',' in value:
        value = value.replace(',',' ')
        return float(value)
    else:
        return float(value)
df['Engine volume'] = df['Engine volume'].apply(convert)
df['Mileage'] = df['Mileage'].apply(convert)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import joblib
import numpy as np


In [7]:
x = df.drop(['Price'], axis=1)
y = df['Price']
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.25)

In [8]:
numeric_features = x_train.select_dtypes(include='float').columns
categorical_features = x_train.select_dtypes(include='float').columns

In [9]:
sc = StandardScaler()
oe = OneHotEncoder(handle_unknown='ignore')

In [10]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num',sc,numeric_features),
        ('cat',oe,categorical_features),
    ]

)

In [18]:
model = make_pipeline(preprocessor, RandomForestRegressor())
model.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Prod. year', 'Engine volume', 'Mileage', 'Cylinders', 'Airbags'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Prod. year', 'Engine volume', 'Mileage', 'Cylinders', 'Airbags'], dtype='object'))])),
                ('randomforestregressor', RandomForestRegressor())])

In [19]:
preds = model.predict(x_test)


In [25]:
mse = mean_squared_error(y_test,preds)
print("MSE:", np.sqrt(mse))


MSE: 12327.521981766826


In [21]:
joblib.dump(model, 'car_pred.joblib')

['car_pred.joblib']

In [22]:
df1 = pd.DataFrame(
    {
        'Manufacturer': ['Mercedes'],
        'Model': ['CLS50'],
        'Prod. year': [2017],
        'Category': ['Sedan'],
        'Fuel type': ['Hybrid'],
        'Engine volume': [8.0],
        'Mileage': [111110],
        'Cylinders': [4.0],
        'Gear box type': ['4x4'],
        'Drive wheels': ['Left wheel'],
        'Color': ['Blue'],
        'Airbags': [4.0]
    }
)

In [23]:
loaded_model = joblib.load('car_pred.joblib')


In [24]:
new_pred = loaded_model.predict(df1)
print("New car's value", new_pred[0])

New car's value 86472.86
